In [1]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from xgboost import plot_importance
from xgboost import cv
import pywt

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import metrics
from scipy import stats
from sklearn.decomposition import PCA
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

In [2]:
'''mean-std noralization'''
def normalization_1(data):
    
    data_mean = np.mean(data,0)   #(72,)
    data_std = np.std(data,0,ddof=1) 
    data_ = (data - data_mean)/data_std
    
    return data_, data_mean , data_std

In [3]:
'''max-min noralization'''
def normalization_2(data):
    
    data_min = np.min(data, 0)
    data_max = np.max(data, 0) 
    data_ = (data - data_min) / (data_max - data_min + 1e-7)
    
    return data_, data_min , data_max

In [4]:
'''one-hot label'''
def to_categorical(y, num_classes=None):
    y = np.array(y, dtype='int')
    input_shape = y.shape
    if input_shape and input_shape[-1] == 1 and len(input_shape) > 1:
        input_shape = tuple(input_shape[:-1])
    y = y.ravel()
    if not num_classes:
        num_classes = np.max(y) + 1
    n = y.shape[0]
    categorical = np.zeros((n, num_classes))
    categorical[np.arange(n), y] = 1
    output_shape = input_shape + (num_classes,)
    categorical = np.reshape(categorical, output_shape)
    return categorical

In [5]:
'''one-hot label'''
# first arg: label
# second arg: num of class 
def one_hot ( labels , Label_class ): 
    one_hot_label = np.array([[ int (i == int (labels[j])) for i in range (Label_class)] for j in range ( len (labels))])      
    return one_hot_label

In [6]:
'''doing wavelet on input data, and spilt them by slidding window'''
def add_window_wavelet(x, time_step, level_):
    
    x_window = []

    for i in range(x.shape[0]):
        series = x[i]
        series_window = []
        for j in range(int(series.shape[0]-time_step)):
            dat = series[j: j+time_step]
            fre_msg_var = []
            for k in range(dat.shape[-1]): #var_num

                coeffs = pywt.wavedec(dat[:,k], 'db1', level = level_, mode='sym')

                fre_msg_ = []
                for i in coeffs: # num_level * 3(mean + std + kurtosis)
                    fre_msg_.append(np.mean(i, 0))
                    fre_msg_.append(np.std(i, 0, ddof=1))
#                     fre_msg_.append(stats.kurtosis(i))
#                     print(f'fre_msg_: {np.mean(i, 0).shape}')
#                     print(f'fre_msg_: {np.std(i, 0, ddof=1).shape}')

                fre_msg_var.append(fre_msg_)
#                 print(f'fre_msg_var: {np.array(fre_msg_var).shape}')

            fre_msg_var = np.array(fre_msg_var)
#             print(f'fre_msg_var: {fre_msg_var.shape}')
            series_window.append(fre_msg_var.reshape([-1]))
#             print(f'series_window: {np.array(series_window).shape}')
        x_window.append(series_window)
        print(f'x_window: {np.array(x_window).shape}')

    return np.array(x_window)

In [7]:
'''spilt labe by slidding window in order to be align with input data'''

def label_add_window(x, time_step):
    
    x_window = []

    for i in range(x.shape[0]):
        series = x[i]
        series_window = []
        for j in range(int(series.shape[0]-time_step)):
            dat = series[j: j+time_step]
            series_window.append(dat[0])
        x_window.append(series_window)
    
    return np.array(x_window)

# Training set

In [8]:
filedir = "tep_train"
filenames = []
train_feature = []
train_lable = []

for filename in os.listdir(filedir):
    filenames.append(os.path.join(filedir,filename))

# print(filenames)
    
filenames = [
'tep_train\\d01.dat', 'tep_train\\d02.dat', 
'tep_train\\d04.dat', 'tep_train\\d06.dat', 
'tep_train\\d07.dat', 'tep_train\\d08.dat',
'tep_train\\d10.dat', 'tep_train\\d11.dat', 'tep_train\\d12.dat', 
'tep_train\\d13.dat', 'tep_train\\d14.dat',
'tep_train\\d17.dat', 'tep_train\\d18.dat', 
'tep_train\\d19.dat', 'tep_train\\d20.dat', 'tep_train\\d21.dat',
'tep_train\\d16.dat', 'tep_train\\d03.dat', 'tep_train\\d15.dat', 'tep_train\\d05.dat',
    
# 'tep_train\\d09.dat'
]
print("num of class, aka the amount of abnormality: ",len(filenames))
print(filenames)

for i in range(len(filenames)):
    tep = np.genfromtxt(filenames[i])
#     print(tep.shape)
#     pca = PCA(n_components = 52)
#     tep = pca.fit_transform(tep)     
#     print(tep.shape)
    train_feature.append(tep)
    
    label = np.ones(tep.shape[0])*i
    train_lable.append(label)
    
train_feature = np.array(train_feature)
train_lable = np.array(train_lable)

print(train_feature.shape)
print(train_lable.shape)

num of class, aka the amount of abnormality:  20
['tep_train\\d01.dat', 'tep_train\\d02.dat', 'tep_train\\d04.dat', 'tep_train\\d06.dat', 'tep_train\\d07.dat', 'tep_train\\d08.dat', 'tep_train\\d10.dat', 'tep_train\\d11.dat', 'tep_train\\d12.dat', 'tep_train\\d13.dat', 'tep_train\\d14.dat', 'tep_train\\d17.dat', 'tep_train\\d18.dat', 'tep_train\\d19.dat', 'tep_train\\d20.dat', 'tep_train\\d21.dat', 'tep_train\\d16.dat', 'tep_train\\d03.dat', 'tep_train\\d15.dat', 'tep_train\\d05.dat']
(20, 480, 52)
(20, 480)


In [9]:
# train_feature[0]

# data_mean = np.mean(train_feature[0],0)   #(72,)
# data_std = np.std(train_feature[0],0,ddof=1) 

# print(data_mean)
# print(data_std)

# Test set


In [10]:
filedir = "tep_test"
filenames_test = []

test_normal = []
test_fault = []
val = []

test_normal_label = []
test_fault_label = []
val_label = []

for filename in os.listdir(filedir):
    filenames_test.append(os.path.join(filedir,filename))
    
# print(filenames_test)

filenames_test = [
'tep_test\\d01_te.dat', 'tep_test\\d02_te.dat', 
'tep_test\\d04_te.dat', 'tep_test\\d06_te.dat', 
'tep_test\\d07_te.dat', 'tep_test\\d08_te.dat',
'tep_test\\d10_te.dat', 'tep_test\\d11_te.dat', 'tep_test\\d12_te.dat', 
'tep_test\\d13_te.dat', 'tep_test\\d14_te.dat',  'tep_test\\d17_te.dat', 'tep_test\\d18_te.dat', 
'tep_test\\d19_te.dat', 'tep_test\\d20_te.dat', 'tep_test\\d21_te.dat',
'tep_test\\d16_te.dat', 'tep_test\\d03_te.dat', 'tep_test\\d15_te.dat', 'tep_test\\d05_te.dat',

# 'tep_test\\d09_te.dat'
]
    
print("num of class, aka the amount of abnormality: ",len(filenames_test))

for i in range(len(filenames_test)):
    
    tep = np.genfromtxt(filenames_test[i])
#     pca = PCA(n_components = 52)
#     tep = pca.fit_transform(tep)    
    
    test_normal_ = tep[:160]
    test_normal.append(test_normal_)

    val_ = tep[160:160+300]
    val.append(val_)
    
    test_fault_ = tep[160+300:]
    test_fault.append(test_fault_)

    label = np.ones(tep.shape[0])*i
    test_normal_label.append(label[:160])
    test_fault_label.append(label[160+300:])
    val_label.append(label[160:160+300])
    
test_normal = np.array(test_normal)
test_fault = np.array(test_fault)
val = np.array(val)
test_normal_label = np.array(test_normal_label)
test_fault_label = np.array(test_fault_label)
val_label = np.array(val_label)

print(test_normal.shape)
print(test_fault.shape)
print(val.shape)
print(test_normal_label.shape)
print(test_fault_label.shape)
print(val_label.shape)

num of class, aka the amount of abnormality:  20
(20, 160, 52)
(20, 500, 52)
(20, 300, 52)
(20, 160)
(20, 500)
(20, 300)


In [11]:
train_lable.shape

(20, 480)

In [14]:
time_step = 110
train_y = label_add_window(train_lable, time_step)#.reshape([-1])
train_y.shape

(20, 370)

In [13]:
time_step = 110
level = 6

train_x = add_window_wavelet(train_feature, time_step, level)
print(train_x.shape)
train_x = train_x.reshape([-1, tep.shape[-1]*((level+1)*2) ])
print(train_x.shape)

# train_x, data_mean , data_std = normalization_1(train_x)
train_y = label_add_window(train_lable, time_step).reshape([-1])

# val_x = add_window_wavelet(val, time_step, level)
# print(val_x.shape)
# val_x = val_x.reshape([-1, tep.shape[-1]*((level+1)*2) ])
# # val_x = (val_x - data_mean)/data_std
# val_y = label_add_window(val_label, time_step).reshape([-1])

# test_normal_x = add_window_wavelet(test_normal, time_step, level)
# test_normal_x = test_normal_x.reshape([-1, tep.shape[-1]*((level+1)*2) ])
# # test_normal_x = (test_normal_x - data_mean)/data_std
# test_normal_y = label_add_window(test_normal_label, time_step).reshape([-1])

# test_fault_x = add_window_wavelet(test_fault, time_step, level)
# test_fault_x = test_fault_x.reshape([-1, tep.shape[-1]*((level+1)*2) ])
# # test_fault_x = (test_fault_x - data_mean)/data_std
# test_fault_y = label_add_window(test_fault_label, time_step).reshape([-1])


print('train_x: ',train_x.shape)
print('train_y: ',train_y.shape)
# print('----------------------------------------------')
# print('val_x: ',val_x.shape)
# print('val_y: ',val_y.shape)
# print('----------------------------------------------')
# print('test_normal_x: ',test_normal_x.shape)
# print('test_normal_y: ',test_normal_y.shape)
# print('----------------------------------------------')
# print('test_fault_x: ',test_fault_x.shape)
# print('test_fault_y: ',test_fault_y.shape)

x_window: (1, 370, 728)
x_window: (2, 370, 728)
x_window: (3, 370, 728)
x_window: (4, 370, 728)
x_window: (5, 370, 728)
x_window: (6, 370, 728)
x_window: (7, 370, 728)
x_window: (8, 370, 728)
x_window: (9, 370, 728)
x_window: (10, 370, 728)
x_window: (11, 370, 728)
x_window: (12, 370, 728)
x_window: (13, 370, 728)
x_window: (14, 370, 728)
x_window: (15, 370, 728)
x_window: (16, 370, 728)
x_window: (17, 370, 728)
x_window: (18, 370, 728)
x_window: (19, 370, 728)
x_window: (20, 370, 728)
(20, 370, 728)
(7400, 728)
train_x:  (7400, 728)
train_y:  (7400,)


In [15]:
52*14

728

In [ ]:
dtrain = xgb.DMatrix(train_x, train_y)
dval = xgb.DMatrix(val_x, val_y)
dtest_normal = xgb.DMatrix(test_normal_x, test_normal_y)
dtest_fault = xgb.DMatrix(test_fault_x, test_fault_y)

In [ ]:
# params = {
# 'objective': 'multi:softprob',
# 'num_class': len(filenames),
# 'seed': 0,
# 'gamma': 0,
# 'max_depth': 10, #10
# # 'random_state': 0,
# 'subsample': 0.075,
# 'colsample_bytree': 0.8,
# 'min_child_weight': 0, #3
# 'lambda': 0.065, # 0.065
# 'grow_policy': 'lossguide',
# 'eta': 0.007,
# 'eval_metric': ['merror'],
# }

# model = xgb.train(params, dtrain, 
#           num_boost_round = 4000, 
#           verbose_eval = 20, 
#           early_stopping_rounds = 200, 
#           evals=[(dtrain, 'train') , (dval, 'valid'), (dtest_normal, 'test_normal'), (dtest_fault, 'test_fault')],
#           )

# print('training finish')

In [ ]:
params = {
'objective': 'multi:softprob',
'num_class': len(filenames),
'seed': 0,
'gamma': 0.5,
'max_depth': 2, #10
# 'random_state': 0,
'subsample': 1,
'min_child_weight': 3,
'lambda': 3,
'grow_policy': 'lossguide',
'eta': 0.007,
'eval_metric': ['merror'],
}

model = xgb.train(params, dtrain, 
          num_boost_round = 4000, 
          verbose_eval = 20, 
          early_stopping_rounds = 200, 
          evals=[(dtrain, 'train') , (dval, 'valid'), (dtest_normal, 'test_normal'), (dtest_fault, 'test_fault')],
          )

print('training finish')

In [ ]:
y_pred = model.predict(xgb.DMatrix(val_x))
yprob = np.argmax(y_pred, axis=1)  # return the index of the biggest pro

predictions = [round(value) for value in yprob]

# evaluate predictions
accuracy = accuracy_score(val_y, predictions)
print("Accuracy: %.5f%%" % (accuracy * 100.0))
print('Recall: %.4f' % metrics.recall_score(val_y, predictions, average='macro'))
print('F1-score: %.4f' % metrics.f1_score(val_y, predictions, average='macro'))
print('Precesion: %.4f' % metrics.precision_score(val_y, predictions, average='macro'))
print("confusion_matrix:")
print(confusion_matrix(val_y, predictions))

In [ ]:
y_pred = model.predict(xgb.DMatrix(test_normal_x))
yprob = np.argmax(y_pred, axis=1)  # return the index of the biggest pro

predictions = [round(value) for value in yprob]

# evaluate predictions
accuracy = accuracy_score(test_normal_y, predictions)
print("Accuracy: %.5f%%" % (accuracy * 100.0))
print('Recall: %.4f' % metrics.recall_score(test_normal_y, predictions, average='macro'))
print('F1-score: %.4f' % metrics.f1_score(test_normal_y, predictions, average='macro'))
print('Precesion: %.4f' % metrics.precision_score(test_normal_y, predictions, average='macro'))
print("confusion_matrix:")
print(confusion_matrix(test_normal_y, predictions))

In [ ]:
y_pred = model.predict(xgb.DMatrix(test_fault_x))
yprob = np.argmax(y_pred, axis=1)  # return the index of the biggest pro

predictions = [round(value) for value in yprob]

# evaluate predictions
accuracy = accuracy_score(test_fault_y, predictions)
print("Accuracy: %.5f%%" % (accuracy * 100.0))
print('Recall: %.4f' % metrics.recall_score(test_fault_y, predictions, average='macro'))
print('F1-score: %.4f' % metrics.f1_score(test_fault_y, predictions, average='macro'))
print('Precesion: %.4f' % metrics.precision_score(test_fault_y, predictions, average='macro'))
print("confusion_matrix:")
print(confusion_matrix(test_fault_y, predictions))

# Feature importance with XGBoost

In [ ]:
xgb.plot_importance(model)
plt.rcParams['figure.figsize'] = [14, 16]
plt.show()

# Fault root tracking

In [ ]:
filedir = "tep_train"
filenames = []
train_feature = []
train_lable = []

normal_tep = np.genfromtxt('tep_train\\d01.dat')#.T[20:]
train_feature.append(normal_tep)

label = np.ones(normal_tep.shape[0])*0
train_lable.append(label)

for filename in os.listdir(filedir):
    filenames.append(os.path.join(filedir,filename))

# print(filenames)
    
filenames = ['tep_train\\d09.dat']
print("num of class: ",len(filenames)+1)
print(filenames)

for i in range(len(filenames)):
    tep = np.genfromtxt(filenames[i])
    train_feature.append(tep)
    
#     tep_mean = np.mean(tep,0)   #(72,)
#     tep_std = np.std(tep,0,ddof=1) 
#     tep = (tep - tep_mean)/tep_std    
    
    label = np.ones(tep.shape[0])*1
    train_lable.append(label)
    
train_feature = np.array(train_feature)
train_lable = np.array(train_lable)

print(train_feature.shape)
print(train_lable.shape)

In [ ]:
# dtw_distance, _ = fastdtw(train_feature[0,:,0], train_feature[1,:,0],
#                           dist=euclidean)

dtw_set = []
for i in range(train_feature.shape[2]):
    dtw_distance, _ = fastdtw(train_feature[0,:,i], train_feature[1,:,i], dist=euclidean)
    dtw_set.append(dtw_distance)



In [ ]:
# print('max ', np.max(dtw_set))
# print('min ', np.min(dtw_set))
# print('mean ', np.mean(dtw_set))
# print('median ', np.median(dtw_set))

1 vs 9

max  23622.77699999998

min  4.160972699999996

mean  2597.777208840384

median  262.61949999999996

3 vs 9

max  13202.499999999985

min  3.434386200000003

mean  818.9624737923073

median  116.13500000000008

15 vs 9

max  15233.199999999999

min  3.2418722999999963

mean  795.0586593711539

median  103.68949999999998

In [ ]:
# 1 vs 9

plt.bar(range(len(dtw_set)), np.sort(dtw_set))
plt.show()

In [ ]:
# 3 vs 9

plt.bar(range(len(dtw_set)), np.sort(dtw_set))
plt.show()

In [ ]:
# 15 vs 9

plt.bar(range(len(dtw_set)), np.sort(dtw_set))
plt.show()

In [ ]:
# for i in range(train_feature.shape[2]):
#     plt.rcParams['figure.figsize'] = [6, 4]
#     plt.figure(i) 
#     plt.title(i)
#     plt.plot(train_feature[0,:,i])
#     plt.plot(train_feature[1,:,i])

In [ ]:
# for i in range(train_feature.shape[2]):
#     plt.rcParams['figure.figsize'] = [6, 4]
#     plt.figure(i) 
#     plt.title(i)
#     plt.plot(train_feature[0,:,i])

In [ ]:
# for i in range(train_feature.shape[2]):
#     plt.rcParams['figure.figsize'] = [6, 4]
#     plt.figure(i) 
#     plt.title(i)
#     plt.plot(train_feature[1,:,i])